# Recommend Lines

Full pipeline of lines recommendation for home team.

In [1]:
%load_ext autoreload

import sys
sys.path.append("/Users/luisd/dev/NHL_stats")
print(sys.path)

['', '/Users/luisd/sw/anaconda3/envs/nhl_stats/lib/python36.zip', '/Users/luisd/sw/anaconda3/envs/nhl_stats/lib/python3.6', '/Users/luisd/sw/anaconda3/envs/nhl_stats/lib/python3.6/lib-dynload', '/Users/luisd/.local/lib/python3.6/site-packages', '/Users/luisd/sw/anaconda3/envs/nhl_stats/lib/python3.6/site-packages', '/Users/luisd/sw/anaconda3/envs/nhl_stats/lib/python3.6/site-packages/IPython/extensions', '/Users/luisd/.ipython', '/Users/luisd/dev/NHL_stats']


In [2]:
import pickle
import datetime
from os import path

import numpy as np

from ReinforcementLearning.NHL.playbyplay.season import Season
from ReinforcementLearning.NHL.player.player_type import PlayerType
from ReinforcementLearning.NHL.lines.category import CategoryFetcher
from ReinforcementLearning.NHL.lines.valuation import QValuesFetcherFromDict, QValuesFetcherFromGameData
from ReinforcementLearning.NHL.lines.recommender import LineRecommender
from ReinforcementLearning.NHL.playbyplay.game import Game, get_lines_for
from ReinforcementLearning.NHL.playbyplay.state_space_data import HockeySS


/Users/luisd/sw/anaconda3/envs/nhl_stats/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
"""Initialization"""
db_root = '/Users/luisd/dev/NHL_stats/data'
repoCode = '/Users/luisd/dev/NHL_stats'

repoModel = path.join(repoCode,
                           'ReinforcementLearning/NHL/playerstats/offVSdef/Automatic_classification/MODEL_perceptron_1layer_10units_relu')

In [4]:
season = Season(db_root=db_root, year_begin=2012, repo_model=repoModel)

# Now lets get game data
base_date = datetime.date(year=2013, month=3, day=13)
result = season.get_game_at_or_just_before(game_date=base_date, home_team_abbr='MTL')
assert (result is not None)
gameId, d = result
print("Fetched game %d, played on %s" % (gameId, d))
data_for_a_game = Game(season, gameId)

INFO:tensorflow:Restoring parameters from /Users/luisd/dev/NHL_stats/ReinforcementLearning/NHL/playerstats/offVSdef/Automatic_classification/MODEL_perceptron_1layer_10units_relu/MODEL_perceptron_1layer_10units_relu
Fetched game 20388, played on 2013-03-13


In [5]:
# prediction of the lines that the 'away' team will use:

formation=get_lines_for(season, base_date, how_many_days_back=5, team_abbrev=data_for_a_game.away_team)
away_lines_names=formation.as_names
away_lines=formation.as_categories

# num_games_back = 5
# ids = season.get_last_n_away_games_since(base_date, n=num_games_back, team_abbrev=data_for_a_game.away_team)
# lines_dict = {}
# for game_id in ids:
#     g = Game(season, gameId=game_id)
#     print("Processing game %s" % (g))
#     result_as_list = g.get_away_lines()
#     for line_as_ids, line_as_types, secs_played in result_as_list:
#         line_as_ids = tuple(map(g.player_id_to_name, line_as_ids))
#         if line_as_ids in lines_dict:
#             # update number of seconds played
#             lines_dict[line_as_ids] = (line_as_types, lines_dict[line_as_ids][1] + secs_played)
#         else:
#             # seed entry in dictionary
#             lines_dict[line_as_ids] = (line_as_types, secs_played)
# print("DONE")

Processing game Season 2012-2013: game 20388
Processing game Season 2012-2013: game 20335
Processing game Season 2012-2013: game 20375
Processing game Season 2012-2013: game 20312
Processing game Season 2012-2013: game 20350
DONE
15 lines used consistently
('7 KYLE TURRIS', '33 JAKOB SILFVERBERG', '14 COLIN GREENING') played 1429.50 secs
('9 MILAN MICHALEK', '15 ZACK SMITH', '11 DANIEL ALFREDSSON') played 1235.50 secs
('25 CHRIS NEIL', '61 MARK STONE', '93 MIKA ZIBANEJAD') played 674.50 secs
('17 MILAN LUCIC', '46 DAVID KREJCI', '8 NATHAN HORTON') played 632.00 secs
('37 PATRICE BERGERON', '63 BRAD MARCHAND', '91 TYLER SEGUIN') played 589.00 secs
('9 GREGORY CAMPBELL', '22 SHAWN THORNTON', '12 DANIEL PAILLE') played 511.50 secs
('25 CHRIS NEIL', '15 ZACK SMITH', '59 DAVE DZIURZYNSKI') played 449.50 secs
("36 JIM O'BRIEN", '12 PETER REGIN', '23 KASPARS DAUGAVINS') played 393.50 secs
('15 ZACK SMITH', '11 DANIEL ALFREDSSON', '73 GUILLAUME LATENDRESSE') played 362.00 secs
('22 ERIK CONDRA

In [6]:
# # for k, v in lines_dict.items():
# #     print(k, v)
# # ok, now sort by seconds played, keep top 4:
# flat_list = list(map(lambda x: (x[0], x[1][0], x[1][1]), lines_dict.items()))
# result_as_list = sorted(flat_list, key=lambda x: x[2], reverse=True)
# print("%d lines used consistently" % (len(result_as_list)))
# for a_line, a_cat, num_secs in result_as_list:
#     print("%s played %.2f secs" % (a_line, num_secs))
# top_4_as_list = result_as_list[:4]
# print("Keeping top 4:")
# for a_line, a_cat, num_secs in top_4_as_list:
#     print("%s played %.2f secs" % (a_line, num_secs))
# away_lines_names = list(map(lambda x: x[0], top_4_as_list)) # as categories
# away_lines = list(map(lambda x: x[1], top_4_as_list)) # as categories

print(away_lines_names)
print(away_lines)

[('7 KYLE TURRIS', '33 JAKOB SILFVERBERG', '14 COLIN GREENING'), ('9 MILAN MICHALEK', '15 ZACK SMITH', '11 DANIEL ALFREDSSON'), ('25 CHRIS NEIL', '61 MARK STONE', '93 MIKA ZIBANEJAD'), ('17 MILAN LUCIC', '46 DAVID KREJCI', '8 NATHAN HORTON')]
[(0, 2, 2), (0, 0, 0), (0, 1, 2), (1, 2, 2)]


In [7]:

# === Now we get the indices in the Q-values tables corresponding to lines

# Line translation table
linedict  = HockeySS(db_root)
linedict.make_line_dictionary()
linedict  = linedict.line_dictionary

# Get lines and translate them
playersCode  =   data_for_a_game.encode_line_players()
linesCode    =   np.array( [[data_for_a_game.recode_line(linedict, a) for a in b] for b in playersCode] )


# Load the Qvalues table
Qvalues = \
pickle.load(open(path.join(repoCode, 'ReinforcementLearning/NHL/playbyplay/data/stable/RL_action_values.p'), 'rb'))[
    'action_values']

# Visualize it dimensions (period x differential x away line's code x home line's code)
print('Q-table dimensions: ', Qvalues.shape)

# Get the Q-value for that specific line
iShift = 0  # First shift
lineShifts = data_for_a_game.lineShifts.as_df(team='both', equal_strength=data_for_a_game.shifts_equal_strength,
                                   regular_time=data_for_a_game.shifts_regular_time, min_duration=20)

player_classes = data_for_a_game.players_classes_mgr.get(equal_strength=True, regular_time=True, min_duration=20, nGames=30) # TODO: why these parameters?
plList = list(player_classes.loc[lineShifts['playersID'].iloc[iShift][0]]['firstlast'].values) + \
         list(player_classes.loc[lineShifts['playersID'].iloc[iShift][1]]['firstlast'].values)
diff = data_for_a_game.recode_differential(lineShifts.iloc[iShift].differential)
period = data_for_a_game.recode_period(lineShifts.iloc[iShift].period)
q_values = Qvalues[period, diff, linesCode[iShift, 0], linesCode[iShift, 1]]
print('[diff = %d, period = %d] First shift: \n\thome team: %s, %s, %s \n\taway team: %s, %s, %s \n\tQvalue: %.2f' % (
diff, period, plList[0], plList[1], plList[2], plList[3], plList[4], plList[5], q_values))

q_values_fetcher_from_game_data = QValuesFetcherFromGameData(game_data=data_for_a_game, lines_dict=linedict, q_values=Qvalues)

line_rec = LineRecommender(
    game=data_for_a_game,
    player_category_fetcher=CategoryFetcher(data_for_game=data_for_a_game),
    q_values_fetcher=q_values_fetcher_from_game_data) # q_values_fetcher_from_tuples)

home_lines_rec = line_rec.recommend_lines_maximize_average(
                                home_team_players_ids=data_for_a_game.get_ids_of_home_players(),
                                away_team_lines = away_lines, examine_max_first_lines=None)
print(home_lines_rec)

print(data_for_a_game.formation_ids_to_str(home_lines_rec))


Q-table dimensions:  (3, 5, 10, 10)
[diff = 2, period = 0] First shift: 
	home team: DAVID DESHARNAIS, BRENDAN GALLAGHER, MAX PACIORETTY 
	away team: KYLE TURRIS, JAKOB SILFVERBERG, COLIN GREENING 
	Qvalue: 3.17
Best fitness: 49.50 by formation 
Line 1: 20 COLBY ARMSTRONG, 27 TRAVIS MOEN, 51 DAVID DESHARNAIS
Line 2: 81 LARS ELLER, 17 MICHAEL RYDER, 37 GABRIEL DUMONT
Line 3: 27 ALEX GALCHENYUK, 11 BRENDAN GALLAGHER, 12 BRIAN GIONTA
Line 4: 25 RYAN WHITE, 67 MAX PACIORETTY, 14 TOMAS PLEKANEC
Best fitness: 49.77 by formation 
Line 1: 20 COLBY ARMSTRONG, 27 TRAVIS MOEN, 51 DAVID DESHARNAIS
Line 2: 81 LARS ELLER, 17 MICHAEL RYDER, 37 GABRIEL DUMONT
Line 3: 12 BRIAN GIONTA, 67 MAX PACIORETTY, 14 TOMAS PLEKANEC
Line 4: 25 RYAN WHITE, 27 ALEX GALCHENYUK, 11 BRENDAN GALLAGHER
Best fitness: 49.86 by formation 
Line 1: 20 COLBY ARMSTRONG, 27 TRAVIS MOEN, 51 DAVID DESHARNAIS
Line 2: 81 LARS ELLER, 17 MICHAEL RYDER, 12 BRIAN GIONTA
Line 3: 37 GABRIEL DUMONT, 27 ALEX GALCHENYUK, 11 BRENDAN GALLAGHER

In [8]:
# let's examine actual decisions and how it compares with optimal:
seconds = {'bad': 0, 'good': 0}
for data in lineShifts[['home_line', 'away_line', 'iceduration']].itertuples():
    home_line = data.home_line
    away_line = data.away_line
    num_seconds = data.iceduration
    away_line_cats = data_for_a_game.classes_of_line(away_line)
    if None in away_line_cats:
        print("Can't get category of one of away players")
    else:
        away_line_cats = tuple(np.sort(away_line_cats))
        if away_line_cats not in away_lines:
            print("%s (categories %s): no optimal calculated" % (away_line, away_line_cats))
        else:
            idx_of_away = away_lines.index(away_line_cats)
            cats_of_optimal = data_for_a_game.classes_of_line(home_lines_rec[idx_of_away])
            home_line_cats = data_for_a_game.classes_of_line(home_line)
            if set(cats_of_optimal) == set(home_line_cats):
                seconds['good'] += num_seconds
            else:
                seconds['bad'] += num_seconds
print(seconds)
print("Home coach's score (in [0,1]) is %.2f" % (seconds['good'] / (seconds['good'] + seconds['bad'])))


[238 621 813] (categories (0, 0, 2)): no optimal calculated
[236 621 813] (categories (0, 0, 2)): no optimal calculated
[238 621 813] (categories (0, 0, 2)): no optimal calculated
[ 228  241 1438] (categories (0, 0, 2)): no optimal calculated
[238 621 813] (categories (0, 0, 2)): no optimal calculated
[ 238  621 1438] (categories (0, 0, 2)): no optimal calculated
[ 228  241 1438] (categories (0, 0, 2)): no optimal calculated
[238 621 813] (categories (0, 0, 2)): no optimal calculated
[238 621 813] (categories (0, 0, 2)): no optimal calculated
[ 228  241 1438] (categories (0, 0, 2)): no optimal calculated
[ 237  871 1438] (categories (0, 0, 2)): no optimal calculated
[ 236  813 1219] (categories (0, 0, 2)): no optimal calculated
[ 237  871 1438] (categories (0, 0, 2)): no optimal calculated
[238 621 813] (categories (0, 0, 2)): no optimal calculated
[238 621 813] (categories (0, 0, 2)): no optimal calculated
[ 237  871 1438] (categories (0, 0, 2)): no optimal calculated
[238 621 813] (c

/Users/luisd/dev/NHL_stats/ReinforcementLearning/NHL/playbyplay/game.py:108: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return list(map(PlayerType.from_int, player_classes.loc[list(a)]["class"].values))


In [9]:
away_lines

[(0, 2, 2), (0, 0, 0), (0, 1, 2), (1, 2, 2)]